# Import required packages

In [3]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F
import os
from urllib import request

# Extracting the required text file

In [4]:
# extracting anna.txt via online
target_url = "https://raw.githubusercontent.com/udacity/deep-learning/master/tensorboard/anna.txt"
data = request.urlopen(target_url)
#print(data)
#print(data) # prints the object id and not file content

# testing the anna.txt loaded file
'''
for line in data:
    print(line)
'''

# create anna.txt and write data to it
f = open("annaKarenina.txt", "w+")
i = 0
for line in data:
    i += 1
    #print(line)
    bytes2Str = line.decode("utf-8")
    #print(bytes2Str)
    f.write(bytes2Str)
#print(i)
f.close()

#Open the file back and read the contents

f = open("annaKarenina.txt", "r")
text = f.read()
#print (text)

# Encoding the characters with integers

In [5]:
# encode the text and map each character to an integer and vice versa
charList = tuple(set(text))
#print(charList)

# dictionary mapping integers to each character
encodeCharWInt = dict(enumerate(charList))
#print(encodeCharWInt)

# look up table mapping character to integers
lookUp = {char: int_ for int_, char in encodeCharWInt.items()}
#print(lookUp)

# encode the text
enArr = np.array([lookUp[char] for char in text])
#print(enArr[:500])

# One hot encoding each element

In [6]:
def oneHotEn(numArr, noOfLabels):
    
    # init the encoded array
    oneHot = np.zeros((np.multiply(*numArr.shape), noOfLabels), dtype=np.float32)
    
    # encoding appropriate elements with 1 after reshaping
    oneHot[np.arange(oneHot.shape[0]), numArr.flatten()] = 1.
    
    # reshape to get original shape back
    oneHot = oneHot.reshape((*numArr.shape, noOfLabels))
    
    return oneHot

# Example of one hot encoding

In [7]:
# check that the function works as expected
test_seq = np.array([[3, 5, 1]])
one_hot = oneHotEn(test_seq, 8)

print(one_hot)

[[[0. 0. 0. 1. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 1. 0. 0.]
  [0. 1. 0. 0. 0. 0. 0. 0.]]]


# Batch splitting algorithm

In [8]:
# function to return batches of size batchSize * seqLen
def splitBatches(numArr, batchSize, seqLen):
    '''
       Arguments
       ---------
       numArr: array you want to make batches from
       batchSize: batch size, the number of sequences per batch
       seqLen: number of encoded chars in a sequence
    '''
    
    totalBatchSize = batchSize * seqLen
    
    # total number of batches we can make, // integer division, round down
    totBatches = len(numArr)//totalBatchSize
    
    # characters truncated upto completed batches
    numArr = numArr[:totBatches * totalBatchSize]
    
    # reshape based on batchSize
    numArr = numArr.reshape((batchSize, -1))
    
    # iterate through the array, one sequence at a time
    for n in range(0, numArr.shape[1], seqLen):
        
        # the input features
        x = numArr[:, n:n+seqLen]
        
        # shifting the input by 1 gives the targets
        y = np.zeros_like(x)
        
        try:
            y[:, :-1], y[:, -1] = x[:, 1:], numArr[:, n+seqLen]
        except IndexError:
            y[:, :-1], y[:, -1] = x[:, 1:], numArr[:, 0]
            
        yield x, y 

# Test batch splitting algorithm

In [10]:
batchTest = splitBatches(enArr, 8, 50)
inputMat, expectedOut = next(batchTest)

# printing out the first 10 items in a sequence
print("Input and output truncted to 10x10")
print('input matrix\n', inputMat[:10, :10])
print('\nexpected output matrix\n', expectedOut[:10, :10])

Input and output truncted to 10x10
input matrix
 [[36 72 56  4  2 82 54 57 13 58]
 [58  2 72 37 64 78 72  2 28 57]
 [64 10 78 57 47 56 10 18 43 58]
 [10 66 57  2 37 57 54 82  2 21]
 [54 14 57 72 82 57 72 56 66 57]
 [37 57 66 21 28 24 64 28 28 21]
 [57 31 37  8  8 74 14 57 66 56]
 [21 10 57  2 72 82 57 24 37 10]]

expected output matrix
 [[72 56  4  2 82 54 57 13 58 58]
 [ 2 72 37 64 78 72  2 28 57 37]
 [10 78 57 47 56 10 18 43 58 58]
 [66 57  2 37 57 54 82  2 21 54]
 [14 57 72 82 57 72 56 66 57 10]
 [57 66 21 28 24 64 28 28 21 37]
 [31 37  8  8 74 14 57 66 56 54]
 [10 57  2 72 82 57 24 37 10 24]]


# Defining the LSTM within a class

In [11]:
class CharLSTM(nn.Module):
    
    def __init__(self, tokens, n_hidden=512, n_layers=4,
                               drop_prob=0.5, lr=0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr
        
        # creating character dictionaries
        self.charList = tokens
        self.encodeCharWInt = dict(enumerate(self.charList))
        self.lookUp = {ch: ii for ii, ch in self.encodeCharWInt.items()}
        
        # define the LSTM
        self.lstm = nn.LSTM(len(self.charList), n_hidden, n_layers, 
                            dropout=drop_prob, batch_first=True)
        
        ## TODO: define a dropout layer
        self.dropout = nn.Dropout(drop_prob)
        
        ## TODO: define the final, fully-connected output layer
        self.fc = nn.Linear(n_hidden, len(self.charList))
      
    # forward pass through the network
    # takes input batch, hidden or cell state as input
    def forward(self, x, hidden):
                
        # get the outputs and the new hidden state from the lstm
        r_op, hidden = self.lstm(x, hidden)
        
        # pass through a dropout layer
        out = self.dropout(r_op)
        
        # Stack up LSTM outputs using view and contiguous to reshape the output
        out = out.contiguous().view(-1, self.n_hidden)
        
        # put x through the fully-connected layer
        out = self.fc(out)
        
        # return the final output and the hidden state
        return out, hidden
    
    
    def initHidden(self, batchSize):
        
        # Create two new tensors with sizes n_layers x batch_size x n_hidden,initialized to zero, for hidden and cell state
        weightTensor = next(self.parameters()).data
        hiddenTensor = (weightTensor.new(self.n_layers, batchSize, self.n_hidden).zero_(),
                      weightTensor.new(self.n_layers, batchSize, self.n_hidden).zero_())
        
        return hiddenTensor

# Function to train the network

In [12]:
def train(net, data, epochs=10, batchSize=10, seqLen=50, lr=0.001, clip=5, valFrac=0.1, printEvery=10):
    ''' 
        Arguments
        ---------
        
        net: CharLSTM network
        data: text data to train the network
        epochs: number of epochs to train
        batchSize: number of mini-sequences per mini-batch, aka batch size
        seqLen: Number of character steps per mini-batch
        lr: learning rate
        clip: gradient clipping
        valFrac: Fraction of data to hold out for validation
        printEvery: Number of steps for printing training and validation loss
    
    '''
    net.train()
    
    optimizer = torch.optim.Adam(net.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    
    # splitting training and validation data
    valID = int(len(data)*(1-valFrac))
    data, valData = data[:valID], data[valID:]
    
    # counts steps through training
    counter = 0
    noOfChars = len(net.charList)
    
    # begin training
    for e in range(epochs):
        
        # initialize hidden state
        h = net.initHidden(batchSize)
        
        for x, y in splitBatches(data, batchSize, seqLen):
            counter += 1
            
            # one-hot encode our data and make them Torch tensors
            x = oneHotEn(x, noOfChars)
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y).type(torch.LongTensor)
            

            # creating new variables for the hidden state, to prevent back-prop through training history
            h = tuple([each.data for each in h])

            # zero accumulated gradients
            net.zero_grad()
            
            # get the output from the model
            output, h = net(inputs, h)
            
            # calculate the loss and perform backprop
            loss = criterion(output, targets.view(batchSize*seqLen))
            loss.backward()
            
            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(net.parameters(), clip)
            optimizer.step()
            
            # loss stats
            if counter % printEvery == 0:
                
                # Get validation loss
                valHid = net.initHidden(batchSize)
                valLosses = []
                net.eval() # set to eval mode
                
                for x, y in splitBatches(valData, batchSize, seqLen):
                    # one-hot encode our data and make them Torch tensors
                    x = oneHotEn(x, noOfChars)
                    x, y = torch.from_numpy(x), torch.from_numpy(y).type(torch.LongTensor)
                    
                    # creating new variables for the hidden state, to prevent back-prop through training history
                    valHid = tuple([each.data for each in valHid])
                    
                    inputs, targets = x, y
                    output, valHid = net(inputs, valHid)
                    valLoss = criterion(output, targets.view(batchSize*seqLen))
                
                    valLosses.append(valLoss.item())
                
                net.train() # reset to train mode after iterationg through validation data
                
                print("Epoch: {}/{}...".format(e+1, epochs),
                      "Step: {}...".format(counter),
                      "Loss: {:.4f}...".format(loss.item()),
                      "Val Loss: {:.4f}".format(np.mean(valLosses)))
                print("**********************************************")

# Network Architecture

In [13]:
# define and print the net
n_hidden=128
n_layers=4

net = CharLSTM(charList, n_hidden, n_layers)
print(net)

CharLSTM(
  (lstm): LSTM(83, 128, num_layers=4, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5)
  (fc): Linear(in_features=128, out_features=83, bias=True)
)


# Begin training the model

In [112]:
batchSize = 64
seqLen = 160 #max length verses
n_epochs = 10 # start smaller if you are just testing initial behavior

# train the model
train(net, enArr, epochs=n_epochs, batchSize=batchSize, seqLen=seqLen, lr=0.001, printEvery=10)

Epoch: 1/10... Step: 10... Loss: 3.9285... Val Loss: 3.7007
**********************************************
Epoch: 1/10... Step: 20... Loss: 3.2850... Val Loss: 3.1731
**********************************************
Epoch: 1/10... Step: 30... Loss: 3.2170... Val Loss: 3.1384
**********************************************
Epoch: 1/10... Step: 40... Loss: 3.1924... Val Loss: 3.1277
**********************************************
Epoch: 1/10... Step: 50... Loss: 3.1811... Val Loss: 3.1273
**********************************************
Epoch: 1/10... Step: 60... Loss: 3.1563... Val Loss: 3.1231
**********************************************
Epoch: 1/10... Step: 70... Loss: 3.1943... Val Loss: 3.1213
**********************************************
Epoch: 1/10... Step: 80... Loss: 3.1550... Val Loss: 3.1215
**********************************************
Epoch: 1/10... Step: 90... Loss: 3.1350... Val Loss: 3.1201
**********************************************
Epoch: 1/10... Step: 100... Loss: 3.1

Epoch: 5/10... Step: 770... Loss: 2.5607... Val Loss: 2.4899
**********************************************
Epoch: 5/10... Step: 780... Loss: 2.5444... Val Loss: 2.4840
**********************************************
Epoch: 5/10... Step: 790... Loss: 2.5523... Val Loss: 2.4767
**********************************************
Epoch: 5/10... Step: 800... Loss: 2.5411... Val Loss: 2.4732
**********************************************
Epoch: 5/10... Step: 810... Loss: 2.5328... Val Loss: 2.4658
**********************************************
Epoch: 5/10... Step: 820... Loss: 2.5066... Val Loss: 2.4692
**********************************************
Epoch: 5/10... Step: 830... Loss: 2.5150... Val Loss: 2.4615
**********************************************
Epoch: 5/10... Step: 840... Loss: 2.5022... Val Loss: 2.4552
**********************************************
Epoch: 5/10... Step: 850... Loss: 2.4849... Val Loss: 2.4512
**********************************************
Epoch: 5/10... Step: 860... 

Epoch: 9/10... Step: 1530... Loss: 2.3317... Val Loss: 2.2337
**********************************************
Epoch: 9/10... Step: 1540... Loss: 2.3381... Val Loss: 2.2259
**********************************************
Epoch: 9/10... Step: 1550... Loss: 2.2865... Val Loss: 2.2253
**********************************************
Epoch: 9/10... Step: 1560... Loss: 2.3165... Val Loss: 2.2231
**********************************************
Epoch: 9/10... Step: 1570... Loss: 2.2872... Val Loss: 2.2202
**********************************************
Epoch: 9/10... Step: 1580... Loss: 2.2869... Val Loss: 2.2156
**********************************************
Epoch: 9/10... Step: 1590... Loss: 2.3045... Val Loss: 2.2149
**********************************************
Epoch: 9/10... Step: 1600... Loss: 2.2935... Val Loss: 2.2153
**********************************************
Epoch: 10/10... Step: 1610... Loss: 2.2901... Val Loss: 2.2103
**********************************************
Epoch: 10/10... St

# Save the trained model

In [1]:
# does not save the model until training is complete
# be careful not to save models with same name
# set model name here and keep track of it
modelName = 'rnnLong.net'

checkpoint = {'n_hidden': net.n_hidden,
              'n_layers': net.n_layers,
              'state_dict': net.state_dict(),
              'tokens': net.charList}

print(net.state_dict())
with open(modelName, 'wb') as f:
    torch.save(checkpoint, f)

NameError: name 'net' is not defined

# Function to predict characters based on the saved model

In [14]:
# function to predict the next character given a particular character
def predict(net, char, hidden=None, topKRes=None):
        
        # tensor inputs
        inputTensor = np.array([[net.lookUp[char]]])
        inputTensor = oneHotEn(inputTensor, len(net.charList))
        inputs = torch.from_numpy(inputTensor)
        
        # detach hidden state from history
        hidden = tuple([each.data for each in hidden])
        
        # get the output of the model
        output, hidden = net(inputs, hidden)

        # get the character probabilities using softmax to get p probabilities
        p = F.softmax(output, dim=1).data
        #print(charProb)
        
        # get top characters considering the k most probable characters
        if topKRes is None:
            topChar = np.arange(len(net.charList))
        else:
            p, topChar = p.topk(topKRes)
            topChar = topChar.numpy().squeeze()
        
        # select the likely next character with some element of randomness
        p = p.numpy().squeeze()
        char = np.random.choice(topChar, p=p/p.sum())
        
        # return the encoded value of the predicted char and the hidden state
        return net.encodeCharWInt[char], hidden

# Evaluation using predictions

In [31]:
# generate samples using the predict function
def sampleOp(net, size, prime='that', topKRes=None):
    
    print("prime = ", prime)
    net.eval() # eval mode
    
    # run through the prime characters
    chars = [ch for ch in prime]
    
    # pass prime chars through the predict function
    hidden = net.initHidden(1)
    for ch in prime:
        predChar, hidden = predict(net, ch, hidden, topKRes=topKRes)

    chars.append(predChar)
    
    # next pass in the previous character and get a new one
    for val in range(size):
        predChar, hidden = predict(net, chars[-1], hidden, topKRes=topKRes)
        chars.append(predChar)

    return ''.join(chars)

# Observing the generated text

In [40]:
# invoking the prediction function
generated = sampleOp(net, 1000, prime='in ', topKRes=5)

# create generatedText.txt and write data to it
f = open("lmGeneratedText.txt", "w+")
for line in generated:
    f.write(line)
f.close()

# open the file back and read the contents
f = open("lmGeneratedText.txt", "r")
genText = f.read()
# print (genText)

in working a the wore thile the seme to she woul she canded. That at the

mele hor had so hin wenl an all whone a the sand ont her, a wose to ald so sore her. And the musher thing the castings ant that the sile the

will thinger. He cell whas she the sould whonh him his sise one the was a tond the pronce, and the wind her sam out of whas

and hen sand, bit of his hind.



"And allyation tore and was worling to her he wam, and., I'd some to white with ander the she to

his sterended the was shing of his antangant ans

him the prace ware. The cones an him wish she tale.



"What ha ment to his with a sould on the could to shile

stonged her had hing alowe of his the wert was have then to saint his some

thine her the said hear that hid hinding, wame,, and wouth though her

think the saminc of his weal wering how the said, and was was hord and he was his and sound than tome home there sinct tarl him and anding the

them. I' wastision,

and, and the was hand hould shere siming that whing h

# Loading a saved model

In [35]:
# enter the name of the model before loading
with open('rnn_2l_256n.net', 'rb') as f:
    checkpoint = torch.load(f)
    
loaded = CharLSTM(checkpoint['tokens'], n_hidden=checkpoint['n_hidden'], n_layers=checkpoint['n_layers'])
loaded.load_state_dict(checkpoint['state_dict'])

# Text generation from a saved model

In [42]:
# sample using a loaded model
generated = sampleOp(loaded, 800, topKRes=5, prime="in ")

# create generatedText.txt and write data to it
f = open("lmGeneratedText.txt", "w+")
for line in generated:
    f.write(line)
f.close()

#Open the file back and read the contents
f = open("lmGeneratedText.txt", "r")
genText = f.read()
#print (genText)

prime =  in 
in time, say, and she say hers of shoughed of that here anl she was to and to sale whit him

hor at here wist the sis as this.



Stay at at wit they walked and ald sall ware him and with his to had and were out at sail, whith a dourd and

tingtare.". The sout her said

hive,

seenter had and he want of sere the with the more of ho hin sith that that the sispirias of her."



"Whing,."



Ald wat at. Bettold ta him sound the canster on assad of thas the shise and

antey hore he wend somented to wely ald the

her wat a tarking, and she contertersed his wat and hav alo the said of her thas here at he and seade the porstel to hin he meassers tat he welt to her she selpelt. He

and hor her and had har to to so tilk. "When we he was a say ther to her would the mart and as and the paster then

the c


In [45]:
mainFile = []
with open('annaKarenina.txt','r') as f:
    for line in f:
        for word in line.split():
           mainFile.append(word)
        
uniqueMain = list(set(mainFile))
#print(uniqueMain)

generatedFile = []
with open('lmGeneratedText.txt','r') as f:
    for line in f:
        for word in line.split():
           generatedFile.append(word)

uniqueGenerate = list(set(generatedFile))
#print(len(uniqueGenerate))
#print(len(mainFile))
#print(len(uniqueMain))

i = 0
for genWord in range(len(uniqueGenerate)):
    for word in range(len(uniqueMain)):
        if uniqueGenerate[genWord] == uniqueMain[word]:
            i += 1
            #print(uniqueGenerate[genWord],uniqueMain[word])
#print(i)
percentAcc = (i/len(uniqueGenerate))*100
print("Text Structure Accuracy: {}%".format(percentAcc))


Text structure Accuracy: 48.421052631578945%
